# **Preparación de los datos para el modelo Machine Learning**

**Proyecto Final: Segunda Entrega**

Este notebook tiene como objetivo preparar los datos para poder realizar el modelo de Machine Learning para la creación de scores para la solución de la problematica de la empresa.

# Cargar los datasets

In [ ]:

# Importar librerías necesarias
from google.colab import drive
import os
import numpy as np
import pandas as pd
from sklearn.preprocessing import OrdinalEncoder

# Montar Google Drive (si no está montado)
mount_point = '/content/drive'

if not os.path.ismount(mount_point):
    drive.mount(mount_point)
    print("Google Drive montado correctamente.")
else:
    print("Google Drive ya estaba montado.")

# Definir la ruta donde están los archivos CSV
data_path = os.path.join(mount_point, "MyDrive", "PROYECTO CIENCIA DE DATOS 2025-2", "dataframes")

# Verificar que el directorio existe
if not os.path.exists(data_path):
    raise FileNotFoundError(f" No se encontró la carpeta en: {data_path}")

# 3Cargar todos los CSV en un diccionario de dataframes
dataframes = {}

for file in os.listdir(data_path):
    if file.endswith(".csv"):
        df_name = file.replace(".csv", "")
        df_path = os.path.join(data_path, file)
        try:
            dataframes[df_name] = pd.read_csv(df_path)
            shape = dataframes[df_name].shape
            print(f"Cargado: {file}  -->  {shape[0]} filas, {shape[1]} columnas")
        except Exception as e:
            print(f"⚠️ Error cargando {file}: {e}")

# Mostrar resumen final
print("\n DataFrames cargados:")
for name, df in dataframes.items():
    print(f"  • {name}: {df.shape}")

print("\n Todos los archivos han sido cargados correctamente.")


Mounted at /content/drive
Google Drive montado correctamente.
Cargado: df_location_filtered.csv  -->  236152 filas, 7 columnas
Cargado: df_industry_filtered.csv  -->  98332 filas, 3 columnas
Cargado: df_cloud_filtered.csv  -->  49510 filas, 2 columnas
Cargado: df_company_tot.csv  -->  34874 filas, 7 columnas
Cargado: df_partners_class_filtered.csv  -->  64549 filas, 2 columnas
Cargado: df_technology_sc_filtered.csv  -->  150451 filas, 2 columnas
Cargado: DatosTransformados.csv  -->  97 filas, 31 columnas
Cargado: df_transformados.csv  -->  97 filas, 31 columnas
Cargado: df_CAT.csv  -->  97 filas, 10 columnas
Cargado: df_Random.csv  -->  97 filas, 25 columnas
Cargado: df_Random_2.csv  -->  597 filas, 28 columnas
Cargado: df_CAT_2.csv  -->  597 filas, 10 columnas
Cargado: df_score.csv  -->  597 filas, 9 columnas

 DataFrames cargados:
  • df_location_filtered: (236152, 7)
  • df_industry_filtered: (98332, 3)
  • df_cloud_filtered: (49510, 2)
  • df_company_tot: (34874, 7)
  • df_partners

# Tratamiento de los datos para el modelo

Como tenemos los datasets divididos, necesitamos unirlos para poder relizar el modelo de ML seleccionado en la entrega anterior y tener en cuenta todo el análisis que hicimos en la entrega pasada ya que estos nos dan las columnas relevantes que vamos a utilizar para el modelo.

In [ ]:
# ==============================================
#  VERIFICACION COLUMNAS EN COMÚN
# ==============================================

# Verificar que haya dataframes cargados
if 'dataframes' not in locals() or len(dataframes) == 0:
    raise ValueError(" No hay dataframes cargados. Ejecuta primero el bloque de carga.")

# Crear un diccionario con el conjunto de columnas de cada DataFrame
column_sets = {name: set(df.columns) for name, df in dataframes.items()}

# Calcular la intersección de todas las columnas
common_columns = set.intersection(*column_sets.values())

# Mostrar resultados
print("COLUMNAS EN COMÚN ENTRE TODOS LOS DATAFRAMES:\n")
if common_columns:
    for col in sorted(common_columns):
        print(f"  • {col}")
else:
    print("⚠️ No hay columnas exactamente iguales en todos los DataFrames.")

# Mostrar las columnas únicas de cada uno (para inspección rápida)
print("\n COLUMNAS POR DATAFRAME:")
for name, cols in column_sets.items():
    print(f"\n{name} ({len(cols)} columnas):")
    print(sorted(cols))

COLUMNAS EN COMÚN ENTRE TODOS LOS DATAFRAMES:

  • Company ID

 COLUMNAS POR DATAFRAME:

df_location_filtered (7 columnas):
['Address Type', 'City', 'Company ID', 'Country', 'Global Region', 'Region', 'State/Province']

df_industry_filtered (3 columnas):
['Company ID', 'Industry Detail (Customer)', 'Sector (Customer)']

df_cloud_filtered (2 columnas):
['Cloud Coverage', 'Company ID']

df_company_tot (7 columnas):
['Company ID', 'Employee Band', 'Number of Locations Band', 'Revenue Band', 'Website', 'Years in Business Band', 'name']

df_partners_class_filtered (2 columnas):
['Company ID', 'Partner Classification']

df_technology_sc_filtered (2 columnas):
['Company ID', 'Technology Scope']

DatosTransformados (31 columnas):
['Clasificacion_Desarrollador', 'Clasificacion_Integrador', 'Clasificacion_Proveedor', 'Clasificacion_Revendedor', 'Cloud_Otros', 'Cloud_Publico', 'Company ID', 'Company Name', 'Employee Band Mod Codificado', 'Global Region_Americas', 'Global Region_EMEA', 'Industria_

Ya teniendo todos los nombres que incluso esta en nuestro diagrama en la entrega anterior, procedemos a unir los datasets con los siguientes criterios:
1) El dataset maestro sera el df_score debido a que aca esta nuestra variable objetivo
2) Gracias al entendimiento de los datos, definimos que hay 8 variables importantes ubicadas en los distintos dfs, a partir de la exploración definimos cual es el mejor valor para no generar duplicados de IDS

Estas serian las 8 variables:
| Prioridad | Variable                     | DataFrame                    |Razon
| --------- | ---------------------------- | ---------------------------- |------------
| 1     | `Revenue Band`               | `df_company_tot`             |Se observó una relación positiva fuerte: empresas con ingresos altos tienen scores más altos y estables. Es una variable determinante principal.
| 2      | `Employee Band`              | `df_company_tot`             |Influye junto con Revenue Band. Las empresas grandes tienen mayor relevancia promedio. También modera el efecto del ingreso (efecto conjunto).
| 3️      | `Industry Detail (Customer)` | `df_industry_filtered`       |Una de las más influyentes: Telecomunicaciones, Educación, Banca, Transporte y Minería = Relevance alto y estable.
| 4️       | `Global Region`              | `df_location_filtered`       |Fuerte concentración en Américas, especialmente EE. UU., México, Brasil y Colombia. La región geográfica influye indirectamente en Relevance a través del tamaño y la adopción tecnológica.
| 5️       | `Cloud Coverage`             | `df_cloud_filtered`          |Muy relevante regionalmente: Américas domina SaaS, lo que se asocia a relevancia estable. Diferencias entre regiones (EMEA, APJ).
| 6️       | `Technology Scope`           | `df_technology_sc_filtered`  |Ligera variación, pero tecnologías emergentes (AR/VR, 3D Printing) muestran Relevance ligeramente más alto.
| 7️       | `Partner Classification`     | `df_partners_class_filtered` |CSP y RSI dominan (alta frecuencia). Representan tipos de partners con orientación cloud; probablemente correlacionan con empresas más maduras.
| 8️       | `Years in Business Band`     | `df_company_tot`             |Moderada influencia: más años tienden a asociarse con mayor estabilidad en Relevance.


### Transformación de las 3 variables que pueden tener un orden.

In [ ]:
# ==============================================
#  Codificacion Estratificada df_company_tot Revenue Band
# ==============================================
df_company_tot = dataframes['df_company_tot']
df_company_tot['Revenue Band Mod'] = (df_company_tot['Revenue Band'].str.replace('K','000')
                                         .str.replace('.5M','500000')
                                         .str.replace('M','000000')
                                         .str.replace('B','000000000')
                                         .str.replace('$','')
                                         .str.replace('+','')
                                         .str.split('-')
                                         .str[0]
                                         .str.replace('<','-') # Para poder organizar, no modifica la codificacion
                                         .astype(float))

valores_unicos_ordenados = sorted(df_company_tot['Revenue Band Mod'].unique())
df_a_codificar = pd.DataFrame(valores_unicos_ordenados, columns=['Revenue Band Mod'])

encoder = OrdinalEncoder()
df_company_tot['Revenue Band Mod Codificado'] = encoder.fit_transform(df_company_tot[['Revenue Band Mod']])

df1 = (df_company_tot.groupby(['Revenue Band', 'Revenue Band Mod','Revenue Band Mod Codificado'])['Company ID'].nunique()
      .reset_index(name='conteo_distinto')
      .sort_values(
          by=['Revenue Band Mod Codificado', 'Revenue Band'], # <--- Lista de columnas para ordenar
          ascending=[True, True] # <--- Descendente para 'conteo', Ascendente para 'id'
      )
      .head(15))
display(df1)

,Revenue Band,Revenue Band Mod,Revenue Band Mod Codificado,conteo_distinto
14,<$100K,-1.000000e+05,0.0,13220
0,$100K-$499K,1.000000e+05,1.0,2287
9,$500K-$999K,5.000000e+05,2.0,4524
5,$1M-$2.49M,1.000000e+06,3.0,5705
6,$2.5M-$4.9M,2.500000e+06,4.0,441
13,$5M-$9.9M,5.000000e+06,5.0,1418
3,$10M-$24.9M,1.000000e+07,6.0,1997
8,$25M-$49M,2.500000e+07,7.0,250
11,$50M-$99M,5.000000e+07,8.0,452
1,$100M-$499M,1.000000e+08,9.0,640


In [ ]:
# ==============================================
#  Codificacion Estratificada df_company_tot Employee Band
# ==============================================

df_company_tot['Employee Band Mod'] = (df_company_tot['Employee Band']
                                         .str.replace('+','')
                                         .str.replace(',','')
                                         .str.split('-')
                                         .str[0]
                                         .astype(float))

valores_unicos_ordenados = sorted(df_company_tot['Employee Band Mod'].unique())
df_a_codificar = pd.DataFrame(valores_unicos_ordenados, columns=['Employee Band Mod'])

encoder = OrdinalEncoder()
encoder_ordenado = OrdinalEncoder(categories=[[valores_unicos_ordenados]])

# Aplicar la transformación

df_company_tot['Employee Band Mod Codificado'] = encoder.fit_transform(df_company_tot[['Employee Band Mod']])

df1 = (df_company_tot.groupby(['Employee Band', 'Employee Band Mod Codificado'])['Company ID'].nunique()
      .reset_index(name='conteo_distinto')
      .sort_values(
          by=['Employee Band Mod Codificado', 'Employee Band'], # <--- Lista de columnas para ordenar
          ascending=[True, True] # <--- Descendente para 'conteo', Ascendente para 'id'
      )
      .head(15))
display(df1)

,Employee Band,Employee Band Mod Codificado,conteo_distinto
1,1-9,0.0,19364
2,10-99,1.0,8357
3,100-499,2.0,2299
4,500-999,3.0,466
0,"1,000+",4.0,1137


In [ ]:
# ==============================================
#  Codificacion Estratificada df_company_tot Years in Business Band
# ==============================================

df_company_tot['Years in Business Band Mod'] = (df_company_tot['Years in Business Band']
                                         .str.replace('+','')
                                         .str.replace(' ','')
                                         .str.replace('<','')
                                         .str.split('-')
                                         .str[0]
                                    .replace('', np.nan)
                                    .astype(float))

valores_unicos_ordenados = sorted(df_company_tot['Years in Business Band Mod'].unique())
df_a_codificar = pd.DataFrame(valores_unicos_ordenados, columns=['Years in Business Band Mod'])

encoder = OrdinalEncoder()

df_company_tot['Years in Business Band Mod Codificado'] = encoder.fit_transform(df_company_tot[['Years in Business Band Mod']])

df1 = (df_company_tot.groupby(['Years in Business Band', 'Years in Business Band Mod Codificado'])['Company ID'].nunique()
      .reset_index(name='conteo_distinto')
      .sort_values(
          by=['Years in Business Band Mod Codificado', 'Years in Business Band'], # <--- Lista de columnas para ordenar
          ascending=[True, True] # <--- Descendente para 'conteo', Ascendente para 'id'
      )
      .head(15))
display(df1)

,Years in Business Band,Years in Business Band Mod Codificado,conteo_distinto
0,00-01,0.0,379
1,02-03,1.0,598
2,04-05,2.0,4419
3,06-08,3.0,2923
4,09-10,4.0,2105
5,11-15,5.0,5217
6,16-20,6.0,4038
7,21-25,7.0,3500
8,26-30,8.0,2531
9,31+,9.0,4169


#Creación de df_master

In [ ]:
# ==============================================
#  BLOQUE: CREACIÓN DE df_master CON VARIABLES ORDINALES
# ==============================================

# 1. Base: df_score -> Company ID + Relevance
df_score = dataframes['df_score'][['Company ID', 'Relevance']].copy()

print("\n--- BASE: df_score ---")
print("Shape:", df_score.shape)
print("Company ID únicos:", df_score['Company ID'].nunique())
display(df_score.head())


# 2. Preparar df_company_tot codificado con las 3 variables ordinales
df_company_tot = dataframes['df_company_tot'].copy()

# Asegurar que Company ID esté bien nombrado
if 'id' in df_company_tot.columns and 'Company ID' not in df_company_tot.columns:
    df_company_tot = df_company_tot.rename(columns={'id': 'Company ID'})

cols_company = [
    'Company ID',
    'Revenue Band Mod Codificado',
    'Employee Band Mod Codificado',
    'Years in Business Band Mod Codificado'
]

df_company_ordinal = df_company_tot[cols_company].copy()

print("\n--- df_company_ordinals ---")
print("Shape:", df_company_ordinal.shape)
print("Company ID únicos:", df_company_ordinal['Company ID'].nunique())
display(df_company_ordinal.head())


# 3. Crear df_master haciendo el primer merge (NO debe duplicar filas)
df_master = pd.merge(
    df_score,
    df_company_ordinal,
    on='Company ID',
    how='left'
)

print("\n--- df_master después del primer JOIN ---")
print("Shape:", df_master.shape)
print("Company ID únicos:", df_master['Company ID'].nunique())
print("Columnas:", df_master.columns.tolist())
display(df_master.head())


# 4. REVISIÓN DE NULOS EN df_master
print("\n--- Revisión de nulos por columna en df_master ---")
print(df_master.isnull().sum())
print("\nTotal de nulos en df_master:", df_master.isnull().sum().sum())




--- BASE: df_score ---
Shape: (597, 2)
Company ID únicos: 597


,Company ID,Relevance
0,111608881,85.81
1,12405281,85.24
2,12794959,85.07
3,112094464,84.80
4,12414034,83.87



--- df_company_ordinals ---
Shape: (34874, 4)
Company ID únicos: 34874


,Company ID,Revenue Band Mod Codificado,Employee Band Mod Codificado,Years in Business Band Mod Codificado
0,12845056,2.0,0.0,4.0
1,15204355,NaN,NaN,NaN
2,111935495,2.0,1.0,6.0
3,107347975,2.0,1.0,7.0
4,118620168,3.0,1.0,4.0



--- df_master después del primer JOIN ---
Shape: (597, 5)
Company ID únicos: 597
Columnas: ['Company ID', 'Relevance', 'Revenue Band Mod Codificado', 'Employee Band Mod Codificado', 'Years in Business Band Mod Codificado']


,Company ID,Relevance,Revenue Band Mod Codificado,Employee Band Mod Codificado,Years in Business Band Mod Codificado
0,111608881,85.81,9.0,4.0,7.0
1,12405281,85.24,3.0,1.0,6.0
2,12794959,85.07,6.0,2.0,8.0
3,112094464,84.80,8.0,2.0,9.0
4,12414034,83.87,6.0,2.0,8.0



--- Revisión de nulos por columna en df_master ---
Company ID                                0
Relevance                                 0
Revenue Band Mod Codificado              20
Employee Band Mod Codificado             20
Years in Business Band Mod Codificado    60
dtype: int64

Total de nulos en df_master: 100


#Transformación de las otras 5 variables antes de hacer el Join

In [ ]:
# ==============================================
#  BLOQUE: AGREGAR GLOBAL REGION (MODA POR COMPANY ID)
# ==============================================

df_location = dataframes['df_location_filtered'][['Company ID', 'Global Region']].copy()

print("\n--- df_location_filtered ---")
print("Shape:", df_location.shape)
print("Company ID únicos:", df_location['Company ID'].nunique())
display(df_location.head())


# 1. Agregación por Company ID usando la moda
df_global_region = (
    df_location
    .groupby('Company ID')['Global Region']
    .agg(lambda x: x.mode().iat[0] if not x.mode().empty else x.iloc[0])
    .reset_index()
)

print("\n--- df_global_region (moda por empresa) ---")
print("Shape:", df_global_region.shape)
print("Company ID únicos:", df_global_region['Company ID'].nunique())
display(df_global_region.head())


# 2. Merge con df_master (NO debe duplicar filas)
print("\nShape de df_master ANTES del merge:", df_master.shape)
print("Company ID únicos ANTES:", df_master['Company ID'].nunique())

df_master = pd.merge(
    df_master,
    df_global_region,
    on='Company ID',
    how='left'
)

print("\nShape de df_master DESPUÉS del merge:", df_master.shape)
print("Company ID únicos DESPUÉS:", df_master['Company ID'].nunique())
print("Columnas actuales:", df_master.columns.tolist())

print("\nNulos en Global Region después del merge:")
print(df_master['Global Region'].isnull().value_counts())




--- df_location_filtered ---
Shape: (236152, 2)
Company ID únicos: 31643


,Company ID,Global Region
0,12824280,Americas
1,92286323,Americas
2,92286323,Americas
3,118648054,Americas
4,107289874,Americas



--- df_global_region (moda por empresa) ---
Shape: (31643, 2)
Company ID únicos: 31643


,Company ID,Global Region
0,12398665,APJ
1,12398761,Americas
2,12398811,Americas
3,12398842,Americas
4,12398874,Americas



Shape de df_master ANTES del merge: (597, 5)
Company ID únicos ANTES: 597

Shape de df_master DESPUÉS del merge: (597, 6)
Company ID únicos DESPUÉS: 597
Columnas actuales: ['Company ID', 'Relevance', 'Revenue Band Mod Codificado', 'Employee Band Mod Codificado', 'Years in Business Band Mod Codificado', 'Global Region']

Nulos en Global Region después del merge:
Global Region
False    596
True       1
Name: count, dtype: int64


In [ ]:
# ==============================================
#  BLOQUE: AGREGAR INDUSTRY (AGRUPADO) POR COMPANY ID
# ==============================================

df_industry = dataframes['df_industry_filtered'][['Company ID', 'Industry Detail (Customer)']].copy()

print("\n--- df_industry_filtered ---")
print("Shape:", df_industry.shape)
print("Company ID únicos:", df_industry['Company ID'].nunique())
display(df_industry.head())


# 1. Crear INDUSTRY_AGRUPADO basado en la letra inicial del código
def map_industry(code):
    if pd.isna(code):
        return np.nan
    code = str(code).strip().upper()

    if code.startswith("B"):
        return "Finanzas"
    elif code.startswith("C"):
        return "Salud"
    elif code.startswith("D"):
        return "Eenergia"   # ojo: double "e" corregimos abajo
    elif code.startswith("E"):
        return "Manufactura"
    elif code.startswith("F"):
        return "Servicios"
    elif code.startswith("G"):
        return "Sector_publico"
    elif code.startswith("H"):
        return "Otros"
    else:
        return "Otros"

# Aplicamos el mapeo real
df_industry["Industry_agrupado"] = df_industry["Industry Detail (Customer)"].apply(map_industry)

# Corrección de "Eenergia" → "Energia"
df_industry["Industry_agrupado"] = df_industry["Industry_agrupado"].replace({"Eenergia": "Energia"})

print("\n--- df_industry con Industry_agrupado ---")
display(df_industry.head())


# 2. Obtener la MODA por empresa
df_industry_moda = (
    df_industry
    .groupby("Company ID")["Industry_agrupado"]
    .agg(lambda x: x.mode().iat[0] if not x.mode().empty else np.nan)
    .reset_index()
)

print("\n--- df_industry_moda (una fila por Company ID) ---")
print("Shape:", df_industry_moda.shape)
print("Company ID únicos:", df_industry_moda['Company ID'].nunique())
display(df_industry_moda.head())


# 3. MERGE con df_master
print("\nShape df_master ANTES del merge:", df_master.shape)
print("Company ID únicos ANTES:", df_master['Company ID'].nunique())

df_master = pd.merge(
    df_master,
    df_industry_moda,
    on='Company ID',
    how='left'
)

print("\nShape df_master DESPUÉS del merge:", df_master.shape)
print("Company ID únicos DESPUÉS:", df_master['Company ID'].nunique())
print("Columnas actuales:", df_master.columns.tolist())

print("\nNulos en Industry_agrupado:")
print(df_master['Industry_agrupado'].isnull().value_counts())



--- df_industry_filtered ---
Shape: (98332, 2)
Company ID únicos: 31643


,Company ID,Industry Detail (Customer)
0,12824280,G1. Education
1,12824280,B1. Banking
2,12824280,B3. Capital Markets
3,12824280,F3. Travel and Transportation
4,12824280,F1. Retail



--- df_industry con Industry_agrupado ---


,Company ID,Industry Detail (Customer),Industry_agrupado
0,12824280,G1. Education,Sector_publico
1,12824280,B1. Banking,Finanzas
2,12824280,B3. Capital Markets,Finanzas
3,12824280,F3. Travel and Transportation,Servicios
4,12824280,F1. Retail,Servicios



--- df_industry_moda (una fila por Company ID) ---
Shape: (31643, 2)
Company ID únicos: 31643


,Company ID,Industry_agrupado
0,12398665,Servicios
1,12398761,Manufactura
2,12398811,Servicios
3,12398842,Servicios
4,12398874,Finanzas



Shape df_master ANTES del merge: (597, 6)
Company ID únicos ANTES: 597

Shape df_master DESPUÉS del merge: (597, 7)
Company ID únicos DESPUÉS: 597
Columnas actuales: ['Company ID', 'Relevance', 'Revenue Band Mod Codificado', 'Employee Band Mod Codificado', 'Years in Business Band Mod Codificado', 'Global Region', 'Industry_agrupado']

Nulos en Industry_agrupado:
Industry_agrupado
False    596
True       1
Name: count, dtype: int64


In [ ]:
# ==============================================
#  Agrupación df_cloud_filtered - Cloud Coverage
# ==============================================
df_cloud_filtered = dataframes['df_cloud_filtered'][['Company ID', 'Cloud Coverage']].copy()

print("\n--- df_cloud_filtered (original) ---")
print("Shape:", df_cloud_filtered.shape)
print("Company ID únicos:", df_cloud_filtered['Company ID'].nunique())
display(df_cloud_filtered.head())

# 1. Recodificación Nube según tu criterio:
#    Publico  = Iaas, SaaS, PaaS
#    Otros    = todo lo demás
df_cloud_filtered['Cloud_agrupado'] = df_cloud_filtered['Cloud Coverage'].apply(
    lambda cloud: 'Publico' if cloud in ['Iaas', 'SaaS', 'PaaS'] else 'Otros'
)

print("\n--- df_cloud_filtered con Cloud_agrupado ---")
display(df_cloud_filtered.head())

# 2. Moda por Company ID (una fila por empresa)
df_cloud_moda = (
    df_cloud_filtered
    .groupby('Company ID')['Cloud_agrupado']
    .agg(lambda x: x.mode().iat[0] if not x.mode().empty else np.nan)
    .reset_index()
)

print("\n--- df_cloud_moda (una fila por Company ID) ---")
print("Shape:", df_cloud_moda.shape)
print("Company ID únicos:", df_cloud_moda['Company ID'].nunique())
display(df_cloud_moda.head())

# 3. Merge con df_master
print("\nShape df_master ANTES del merge Cloud:", df_master.shape)
print("Company ID únicos ANTES:", df_master['Company ID'].nunique())

df_master = pd.merge(
    df_master,
    df_cloud_moda,
    on='Company ID',
    how='left'
)

print("\nShape df_master DESPUÉS del merge Cloud:", df_master.shape)
print("Company ID únicos DESPUÉS:", df_master['Company ID'].nunique())
print("Columnas actuales:", df_master.columns.tolist())

print("\nNulos en Cloud_agrupado:")
print(df_master['Cloud_agrupado'].isnull().value_counts())



--- df_cloud_filtered (original) ---
Shape: (49510, 2)
Company ID únicos: 31643


,Company ID,Cloud Coverage
0,12824280,SaaS
1,12824280,Other Cloud: Hybrid/Private/On-prem virtualiza...
2,92286323,SaaS
3,92286323,Other Cloud: Hybrid/Private/On-prem virtualiza...
4,118648054,-



--- df_cloud_filtered con Cloud_agrupado ---


,Company ID,Cloud Coverage,Cloud_agrupado
0,12824280,SaaS,Publico
1,12824280,Other Cloud: Hybrid/Private/On-prem virtualiza...,Otros
2,92286323,SaaS,Publico
3,92286323,Other Cloud: Hybrid/Private/On-prem virtualiza...,Otros
4,118648054,-,Otros



--- df_cloud_moda (una fila por Company ID) ---
Shape: (31643, 2)
Company ID únicos: 31643


,Company ID,Cloud_agrupado
0,12398665,Publico
1,12398761,Otros
2,12398811,Otros
3,12398842,Publico
4,12398874,Otros



Shape df_master ANTES del merge Cloud: (597, 7)
Company ID únicos ANTES: 597

Shape df_master DESPUÉS del merge Cloud: (597, 8)
Company ID únicos DESPUÉS: 597
Columnas actuales: ['Company ID', 'Relevance', 'Revenue Band Mod Codificado', 'Employee Band Mod Codificado', 'Years in Business Band Mod Codificado', 'Global Region', 'Industry_agrupado', 'Cloud_agrupado']

Nulos en Cloud_agrupado:
Cloud_agrupado
False    596
True       1
Name: count, dtype: int64


In [ ]:
# ==============================================
#  Agrupación df_technology_sc_filtered - Technology Scope
# ==============================================
df_tech = dataframes['df_technology_sc_filtered'][['Company ID', 'Technology Scope']].copy()

print("\n--- df_technology_sc_filtered (original) ---")
print("Shape:", df_tech.shape)
print("Company ID únicos:", df_tech['Company ID'].nunique())
display(df_tech.head())

# 1. Definir grupos según tu esquema
infraestructura = ['Mobility', 'IoT', 'Cloud']
inteligencia    = ['Big Data and Analytics', 'AI', 'Robotics']
usuario         = ['AR/VR', '3D Printing', 'Social']
seguridad       = ['Security', 'Blockchain']

def map_tech(scope):
    if pd.isna(scope):
        return np.nan
    s = str(scope).strip()

    if s in infraestructura:
        return 'Infraestructura'
    elif s in inteligencia:
        return 'Inteligencia'
    elif s in usuario:
        return 'Usuario'
    elif s in seguridad:
        return 'Seguridad'
    elif s == '-':
        return 'Otros'
    else:
        return 'Otros'

# 2. Crear columna agrupada
df_tech['Technology_agrupado'] = df_tech['Technology Scope'].apply(map_tech)

print("\n--- df_tech con Technology_agrupado ---")
display(df_tech.head())

# 3. Moda por Company ID (una tecnología agrupada dominante por empresa)
df_tech_moda = (
    df_tech
    .groupby('Company ID')['Technology_agrupado']
    .agg(lambda x: x.mode().iat[0] if not x.mode().empty else np.nan)
    .reset_index()
)

print("\n--- df_tech_moda (una fila por Company ID) ---")
print("Shape:", df_tech_moda.shape)
print("Company ID únicos:", df_tech_moda['Company ID'].nunique())
display(df_tech_moda.head())

# 4. Merge con df_master
print("\nShape df_master ANTES del merge Technology:", df_master.shape)
print("Company ID únicos ANTES:", df_master['Company ID'].nunique())

df_master = pd.merge(
    df_master,
    df_tech_moda,
    on='Company ID',
    how='left'
)

print("\nShape df_master DESPUÉS del merge Technology:", df_master.shape)
print("Company ID únicos DESPUÉS:", df_master['Company ID'].nunique())
print("Columnas actuales:", df_master.columns.tolist())

print("\nNulos en Technology_agrupado:")
print(df_master['Technology_agrupado'].isnull().value_counts())




--- df_technology_sc_filtered (original) ---
Shape: (150451, 2)
Company ID únicos: 31643


,Company ID,Technology Scope
0,12824280,Mobility
1,12824280,Big Data and Analytics
2,12824280,AI
3,12824280,Social
4,12824280,IoT



--- df_tech con Technology_agrupado ---


,Company ID,Technology Scope,Technology_agrupado
0,12824280,Mobility,Infraestructura
1,12824280,Big Data and Analytics,Inteligencia
2,12824280,AI,Inteligencia
3,12824280,Social,Usuario
4,12824280,IoT,Infraestructura



--- df_tech_moda (una fila por Company ID) ---
Shape: (31643, 2)
Company ID únicos: 31643


,Company ID,Technology_agrupado
0,12398665,Infraestructura
1,12398761,Inteligencia
2,12398811,Infraestructura
3,12398842,Infraestructura
4,12398874,Infraestructura



Shape df_master ANTES del merge Technology: (597, 8)
Company ID únicos ANTES: 597

Shape df_master DESPUÉS del merge Technology: (597, 9)
Company ID únicos DESPUÉS: 597
Columnas actuales: ['Company ID', 'Relevance', 'Revenue Band Mod Codificado', 'Employee Band Mod Codificado', 'Years in Business Band Mod Codificado', 'Global Region', 'Industry_agrupado', 'Cloud_agrupado', 'Technology_agrupado']

Nulos en Technology_agrupado:
Technology_agrupado
False    596
True       1
Name: count, dtype: int64


In [ ]:
# ==============================================
#  Agrupación df_partners_class_filtered - Partner Classification
# ==============================================
df_partner = dataframes['df_partners_class_filtered'][['Company ID', 'Partner Classification']].copy()

print("\n--- df_partners_class_filtered (original) ---")
print("Shape:", df_partner.shape)
print("Company ID únicos:", df_partner['Company ID'].nunique())
display(df_partner.head())

# DEFINICIÓN DE GRUPOS
desarrollador = ['Independent Software Vendor (ISV)']
integrador    = ['Regional System Integrator (RSI)', 'Global Systems Integrator (GSI)']
proveedor     = ['Cloud Service Provider (CSP)', 'Managed Service Provider (MSP)']
revendedor    = ['Direct Market Reseller (DMR)', 'Value Added Reseller (VAR)', 'Distributor']

def map_partner(p):
    if pd.isna(p):
        return np.nan
    s = str(p).strip()

    if s in desarrollador:
        return "Desarrollador"
    elif s in integrador:
        return "Integrador"
    elif s in proveedor:
        return "Proveedor"
    elif s in revendedor:
        return "Revendedor"
    elif s == '-':
        return "Otros"
    else:
        return "Otros"

# 1. Crear columna agrupada
df_partner['Partner_agrupado'] = df_partner['Partner Classification'].apply(map_partner)

print("\n--- df_partner con Partner_agrupado ---")
display(df_partner.head())

# 2. Moda por Company ID (una sola categoría por empresa)
df_partner_moda = (
    df_partner
    .groupby('Company ID')['Partner_agrupado']
    .agg(lambda x: x.mode().iat[0] if not x.mode().empty else np.nan)
    .reset_index()
)

print("\n--- df_partner_moda (una fila por Company ID) ---")
print("Shape:", df_partner_moda.shape)
print("Company ID únicos:", df_partner_moda['Company ID'].nunique())
display(df_partner_moda.head())

# 3. Merge con df_master
print("\nShape df_master ANTES del merge Partner:", df_master.shape)
print("Company ID únicos ANTES:", df_master['Company ID'].nunique())

df_master = pd.merge(
    df_master,
    df_partner_moda,
    on='Company ID',
    how='left'
)

print("\nShape df_master DESPUÉS del merge Partner:", df_master.shape)
print("Company ID únicos DESPUÉS:", df_master['Company ID'].nunique())
print("Columnas actuales:", df_master.columns.tolist())

print("\nNulos en Partner_agrupado:")
print(df_master['Partner_agrupado'].isnull().value_counts())




--- df_partners_class_filtered (original) ---
Shape: (64549, 2)
Company ID únicos: 31643


,Company ID,Partner Classification
0,12824280,Independent Software Vendor (ISV)
1,12824280,Regional System Integrator (RSI)
2,12824280,Cloud Service Provider (CSP)
3,92286323,Cloud Service Provider (CSP)
4,118648054,Direct Market Reseller (DMR)



--- df_partner con Partner_agrupado ---


,Company ID,Partner Classification,Partner_agrupado
0,12824280,Independent Software Vendor (ISV),Desarrollador
1,12824280,Regional System Integrator (RSI),Integrador
2,12824280,Cloud Service Provider (CSP),Proveedor
3,92286323,Cloud Service Provider (CSP),Proveedor
4,118648054,Direct Market Reseller (DMR),Revendedor



--- df_partner_moda (una fila por Company ID) ---
Shape: (31643, 2)
Company ID únicos: 31643


,Company ID,Partner_agrupado
0,12398665,Integrador
1,12398761,Integrador
2,12398811,Integrador
3,12398842,Integrador
4,12398874,Integrador



Shape df_master ANTES del merge Partner: (597, 9)
Company ID únicos ANTES: 597

Shape df_master DESPUÉS del merge Partner: (597, 10)
Company ID únicos DESPUÉS: 597
Columnas actuales: ['Company ID', 'Relevance', 'Revenue Band Mod Codificado', 'Employee Band Mod Codificado', 'Years in Business Band Mod Codificado', 'Global Region', 'Industry_agrupado', 'Cloud_agrupado', 'Technology_agrupado', 'Partner_agrupado']

Nulos en Partner_agrupado:
Partner_agrupado
False    596
True       1
Name: count, dtype: int64


In [ ]:
print("\n--- Nulos por columna en df_master ---")
display(df_master.isnull().sum())



--- Nulos por columna en df_master ---


,0
Company ID,0
Relevance,0
Revenue Band Mod Codificado,20
Employee Band Mod Codificado,20
Years in Business Band Mod Codificado,60
Global Region,1
Industry_agrupado,1
Cloud_agrupado,1
Technology_agrupado,1
Partner_agrupado,1


Imputamos nulls con la mediana para revenue, employee y Years. Mientras que en los demas lo llenamos con la categoria Otros.

In [ ]:
# ==============================================
#  IMPUTACIÓN DE NULOS EN df_master
# ==============================================

df_clean = df_master.copy()

# 1. Rellenar ordinales con mediana
ordinales = [
    'Revenue Band Mod Codificado',
    'Employee Band Mod Codificado',
    'Years in Business Band Mod Codificado'
]

for col in ordinales:
    mediana = df_clean[col].median()
    df_clean[col] = df_clean[col].fillna(mediana)
    print(f"Imputada mediana en {col}: {mediana}")


# 2. Rellenar categóricas con 'Otros'
categoricas = [
    'Global Region',
    'Industry_agrupado',
    'Cloud_agrupado',
    'Technology_agrupado',
    'Partner_agrupado'
]

for col in categoricas:
    df_clean[col] = df_clean[col].fillna('Otros')
    print(f"Imputados nulos en {col} con 'Otros'")


# 3. Verificar
print("\n--- Nulos por columna después de imputación ---")
print(df_clean.isnull().sum())


Imputada mediana en Revenue Band Mod Codificado: 2.0
Imputada mediana en Employee Band Mod Codificado: 0.0
Imputada mediana en Years in Business Band Mod Codificado: 5.0
Imputados nulos en Global Region con 'Otros'
Imputados nulos en Industry_agrupado con 'Otros'
Imputados nulos en Cloud_agrupado con 'Otros'
Imputados nulos en Technology_agrupado con 'Otros'
Imputados nulos en Partner_agrupado con 'Otros'

--- Nulos por columna después de imputación ---
Company ID                               0
Relevance                                0
Revenue Band Mod Codificado              0
Employee Band Mod Codificado             0
Years in Business Band Mod Codificado    0
Global Region                            0
Industry_agrupado                        0
Cloud_agrupado                           0
Technology_agrupado                      0
Partner_agrupado                         0
dtype: int64


Ahora vamos a dividir en dos el df_master para aplicara 2 modelos de regresión.

In [ ]:
# ==============================================
#  PREPARACIÓN RANDOM FOREST - ONE HOT ENCODING
# ==============================================

df_clean_2 = df_clean.copy()

categorical_cols = [
    'Global Region',
    'Industry_agrupado',
    'Cloud_agrupado',
    'Technology_agrupado',
    'Partner_agrupado'
]

df_master_rf = pd.get_dummies(
    df_clean_2,
    columns=categorical_cols,
    drop_first=False
)

# --- 🔥 Convertir booleanos a enteros (0/1) ---
bool_cols = df_master_rf.select_dtypes(include=['bool']).columns
df_master_rf[bool_cols] = df_master_rf[bool_cols].astype(int)

print("\n--- Columnas convertidas de True/False a 0/1 ---")
print(list(bool_cols))


# --- Crear X_rf y y_rf ---
y_rf = df_master_rf['Relevance']
X_rf = df_master_rf.drop(['Relevance', 'Company ID'], axis=1)

print("\n--- Dimensiones X_rf y y_rf ---")
print("X_rf:", X_rf.shape)
print("y_rf:", y_rf.shape)

# ==============================================
# VISUALIZACIÓN DEL DATASET FINAL (df_master_rf)
# ==============================================

print("\n--- DataFrame final para Random Forest (df_master_rf) ---")
print("Shape:", df_master_rf.shape)

# Mostrar 5 filas completas (cuidado si hay muchas columnas)
display(df_master_rf.head(5))




--- Columnas convertidas de True/False a 0/1 ---
['Global Region_APJ', 'Global Region_Americas', 'Global Region_EMEA', 'Global Region_Otros', 'Industry_agrupado_Energia', 'Industry_agrupado_Finanzas', 'Industry_agrupado_Manufactura', 'Industry_agrupado_Otros', 'Industry_agrupado_Salud', 'Industry_agrupado_Sector_publico', 'Industry_agrupado_Servicios', 'Cloud_agrupado_Otros', 'Cloud_agrupado_Publico', 'Technology_agrupado_Infraestructura', 'Technology_agrupado_Inteligencia', 'Technology_agrupado_Otros', 'Technology_agrupado_Seguridad', 'Technology_agrupado_Usuario', 'Partner_agrupado_Desarrollador', 'Partner_agrupado_Integrador', 'Partner_agrupado_Otros', 'Partner_agrupado_Proveedor', 'Partner_agrupado_Revendedor']

--- Dimensiones X_rf y y_rf ---
X_rf: (597, 26)
y_rf: (597,)

--- DataFrame final para Random Forest (df_master_rf) ---
Shape: (597, 28)


,Company ID,Relevance,Revenue Band Mod Codificado,Employee Band Mod Codificado,Years in Business Band Mod Codificado,Global Region_APJ,Global Region_Americas,Global Region_EMEA,Global Region_Otros,Industry_agrupado_Energia,...,Technology_agrupado_Infraestructura,Technology_agrupado_Inteligencia,Technology_agrupado_Otros,Technology_agrupado_Seguridad,Technology_agrupado_Usuario,Partner_agrupado_Desarrollador,Partner_agrupado_Integrador,Partner_agrupado_Otros,Partner_agrupado_Proveedor,Partner_agrupado_Revendedor
0,111608881,85.81,9.0,4.0,7.0,0,1,0,0,1,...,0,1,0,0,0,0,1,0,0,0
1,12405281,85.24,3.0,1.0,6.0,0,1,0,0,0,...,1,0,0,0,0,1,0,0,0,0
2,12794959,85.07,6.0,2.0,8.0,1,0,0,0,0,...,1,0,0,0,0,0,1,0,0,0
3,112094464,84.80,8.0,2.0,9.0,0,0,1,0,0,...,0,0,1,0,0,1,0,0,0,0
4,12414034,83.87,6.0,2.0,8.0,0,1,0,0,0,...,1,0,0,0,0,0,1,0,0,0


In [ ]:
import os

# Directorio donde se guardan los archivos
output_dir = '/content/drive/MyDrive/PROYECTO CIENCIA DE DATOS 2025-2/dataframes'
os.makedirs(output_dir, exist_ok=True)

# 1. Guardar dataset para CatBoost
df_CAT = df_clean.copy()
output_path_cat = os.path.join(output_dir, 'df_CAT.csv')
df_CAT.to_csv(output_path_cat, index=False)
print(f'Guardado correctamente: {output_path_cat}')

# 2. Guardar dataset para Random Forest
df_Random = df_master_rf.copy()
output_path_rf = os.path.join(output_dir, 'df_Random.csv')
df_Random.to_csv(output_path_rf, index=False)
print(f'Guardado correctamente: {output_path_rf}')


Guardado correctamente: /content/drive/MyDrive/PROYECTO CIENCIA DE DATOS 2025-2/dataframes/df_CAT.csv
Guardado correctamente: /content/drive/MyDrive/PROYECTO CIENCIA DE DATOS 2025-2/dataframes/df_Random.csv
